# Nordic Retail Delivery Analysis

## What this project is about
This project simulates order and delivery data for a Nordic retail business
operating in Copenhagen, Aarhus, and Odense.

The goal is to understand how revenue and delivery performance behave
over time and across regions.

## Questions this data helps answer
- Are orders delivered on time?
- Do delivery delays increase during certain months?
- How do regions compare in revenue and delivery reliability?

## What this notebook covers
- Creating realistic retail data
- Adding delivery delays to reflect real-world conditions
- Validating the fields used later in Power BI

Dashboards and insights are built separately in Power BI.


## What this project does

This notebook creates and validates a small, realistic dataset for a Nordic retail business operating in Denmark.

The focus is on delivery performance and revenue:
- Are orders delivered on time?
- How does on-time performance change by month?
- How do Danish regions compare in both revenue and reliability?

The cleaned and aggregated outputs from this notebook are exported to CSV and used directly in a Power BI dashboard.

In [1]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import random

In [2]:
conn = sqlite3.connect("nordic_retail.db")

In [3]:
random.seed(42)

customers = []
for customer_id in range(1, 301):
    signup_date = datetime(2025, 1, 1) + timedelta(days=random.randint(0, 180))
    customers.append([customer_id, signup_date.date()])

customers_df = pd.DataFrame(customers, columns=["customer_id", "signup_date"])
customers_df.to_sql("customers", conn, if_exists="replace", index=False)

300

In [4]:
regions = ["Copenhagen", "Aarhus", "Odense"]

orders = []
order_id = 1

for _ in range(800):
    customer_id = random.randint(1, 300)
    order_date = datetime(2025, 7, 1) + timedelta(days=random.randint(0, 180))
    promised_delivery = order_date + timedelta(days=3)
    
    # Introduce delays (more likely in Aug–Sep)
    delay_days = 0
    if order_date.month in [8, 9] and random.random() < 0.25:
        delay_days = random.randint(1, 3)
    elif random.random() < 0.1:
        delay_days = random.randint(1, 2)
    
    actual_delivery = promised_delivery + timedelta(days=delay_days)
    order_value = random.randint(200, 1200)
    
    orders.append([
        order_id,
        customer_id,
        order_date.date(),
        random.choice(regions),
        promised_delivery.date(),
        actual_delivery.date(),
        order_value
    ])
    
    order_id += 1

orders_df = pd.DataFrame(
    orders,
    columns=[
        "order_id",
        "customer_id",
        "order_date",
        "region",
        "promised_delivery_date",
        "actual_delivery_date",
        "order_value"
    ]
)

orders_df.to_sql("orders", conn, if_exists="replace", index=False)

800

In [5]:
categories = ["Rings", "Bracelets", "Necklaces"]

items = []
for order in orders_df.itertuples():
    category = random.choice(categories)
    items.append([order.order_id, category, order.order_value])

items_df = pd.DataFrame(items, columns=["order_id", "category", "item_value"])
items_df.to_sql("order_items", conn, if_exists="replace", index=False)

800

In [6]:
pd.read_sql("""
SELECT
    order_id,
    region,
    order_date,
    promised_delivery_date,
    actual_delivery_date,
    CASE 
        WHEN actual_delivery_date <= promised_delivery_date THEN 1
        ELSE 0
    END AS on_time
FROM orders
LIMIT 10;
""", conn)

,order_id,region,order_date,promised_delivery_date,actual_delivery_date,on_time
0,1,Aarhus,2025-10-08,2025-10-11,2025-10-11,1
1,2,Odense,2025-10-17,2025-10-20,2025-10-20,1
2,3,Copenhagen,2025-08-09,2025-08-12,2025-08-13,0
3,4,Copenhagen,2025-11-16,2025-11-19,2025-11-21,0
4,5,Odense,2025-10-31,2025-11-03,2025-11-03,1
5,6,Copenhagen,2025-07-15,2025-07-18,2025-07-18,1
6,7,Copenhagen,2025-11-30,2025-12-03,2025-12-04,0
7,8,Odense,2025-09-02,2025-09-05,2025-09-06,0
8,9,Aarhus,2025-11-22,2025-11-25,2025-11-25,1
9,10,Aarhus,2025-12-19,2025-12-22,2025-12-22,1


In [7]:
pd.read_sql("SELECT COUNT(*) AS orders FROM orders", conn)

,orders
0,800


In [8]:
pd.read_sql("""
SELECT
    ROUND(AVG(
        CASE 
            WHEN actual_delivery_date <= promised_delivery_date THEN 1.0
            ELSE 0.0
        END
    ), 3) AS on_time_rate
FROM orders;
""", conn)

,on_time_rate
0,0.834


In [9]:
pd.read_sql("""
SELECT
    strftime('%Y-%m', order_date) AS month,
    ROUND(AVG(
        CASE 
            WHEN actual_delivery_date <= promised_delivery_date THEN 1.0
            ELSE 0.0
        END
    ), 3) AS on_time_rate
FROM orders
GROUP BY month
ORDER BY month;
""", conn)

,month,on_time_rate
0,2025-07,0.893
1,2025-08,0.706
2,2025-09,0.667
3,2025-10,0.926
4,2025-11,0.873
5,2025-12,0.919


In [10]:
pd.read_sql("""
SELECT
    region,
    ROUND(AVG(
        CASE 
            WHEN actual_delivery_date <= promised_delivery_date THEN 1.0
            ELSE 0.0
        END
    ), 3) AS on_time_rate
FROM orders
GROUP BY region
ORDER BY on_time_rate DESC;
""", conn)

,region,on_time_rate
0,Copenhagen,0.846
1,Aarhus,0.832
2,Odense,0.824


In [11]:
pd.read_sql("""
SELECT
    region,
    COUNT(*) AS orders,
    ROUND(SUM(order_value), 0) AS total_revenue,
    ROUND(AVG(
        CASE 
            WHEN actual_delivery_date <= promised_delivery_date THEN 1.0
            ELSE 0.0
        END
    ), 3) AS on_time_rate
FROM orders
GROUP BY region
ORDER BY total_revenue DESC;
""", conn)

,region,orders,total_revenue,on_time_rate
0,Aarhus,292,205724.0,0.832
1,Odense,261,184911.0,0.824
2,Copenhagen,247,161397.0,0.846


## Key observations (from the data)

- Delivery performance drops noticeably in August and September, reflecting seasonal delays.
- Copenhagen shows the highest on-time rate overall, despite handling fewer orders than Aarhus.
- Revenue and delivery reliability do not always move together, highlighting trade-offs between volume and service quality.

Detailed exploration of these patterns is done in the Power BI dashboard.

## Final Notes

The aggregated datasets created in this notebook
(on-time rate by month, on-time rate by region, and revenue by region)
were exported to CSV files.

These CSVs were then used to build an interactive Power BI dashboard.

This notebook focuses only on:
- Creating realistic retail data
- Validating delivery performance logic
- Preparing clean inputs for visualization

All analysis and insights are explored visually in Power BI.
